## Ejemplo 1: Problema de la Mochila 0/1

Resuelve la mochila usando cotas para podar el árbol de búsqueda.

In [ ]:
import heapq

class Nodo:
    def __init__(self, nivel, peso, valor, incluidos):
        self.nivel = nivel
        self.peso = peso
        self.valor = valor
        self.incluidos = incluidos
        self.cota = 0
    
    def __lt__(self, otro):
        return self.cota > otro.cota  # Max heap

def mochila_branch_bound(pesos, valores, capacidad):
    """
    Mochila 0/1 usando Branch and Bound.
    Usa cotas fraccionarias para poda.
    """
    n = len(pesos)
    
    # Ordenar por ratio valor/peso
    items = sorted(range(n), key=lambda i: valores[i]/pesos[i], reverse=True)
    
    def calcular_cota_superior(nodo):
        """Calcula cota usando enfoque voraz fraccionario."""
        if nodo.peso >= capacidad:
            return 0
        
        cota = nodo.valor
        nivel = nodo.nivel + 1
        peso_total = nodo.peso
        
        # Agregar items completos
        while nivel < n and peso_total + pesos[items[nivel]] <= capacidad:
            peso_total += pesos[items[nivel]]
            cota += valores[items[nivel]]
            nivel += 1
        
        # Agregar fracción del siguiente
        if nivel < n:
            cota += (capacidad - peso_total) * (valores[items[nivel]] / pesos[items[nivel]])
        
        return cota
    
    # Inicializar
    mejor_valor = 0
    mejor_solucion = []
    
    raiz = Nodo(-1, 0, 0, [])
    raiz.cota = calcular_cota_superior(raiz)
    pq = [raiz]
    nodos_explorados = 0
    nodos_podados = 0
    
    while pq:
        nodo = heapq.heappop(pq)
        nodos_explorados += 1
        
        # Poda
        if nodo.cota <= mejor_valor:
            nodos_podados += 1
            continue
        
        if nodo.nivel == n - 1:
            if nodo.valor > mejor_valor:
                mejor_valor = nodo.valor
                mejor_solucion = nodo.incluidos[:]
            continue
        
        nivel = nodo.nivel + 1
        idx = items[nivel]
        
        # Rama: incluir item
        if nodo.peso + pesos[idx] <= capacidad:
            incluir = Nodo(
                nivel,
                nodo.peso + pesos[idx],
                nodo.valor + valores[idx],
                nodo.incluidos + [idx]
            )
            incluir.cota = calcular_cota_superior(incluir)
            
            if incluir.cota > mejor_valor:
                if incluir.valor > mejor_valor:
                    mejor_valor = incluir.valor
                    mejor_solucion = incluir.incluidos[:]
                heapq.heappush(pq, incluir)
        
        # Rama: no incluir item
        no_incluir = Nodo(nivel, nodo.peso, nodo.valor, nodo.incluidos[:])
        no_incluir.cota = calcular_cota_superior(no_incluir)
        
        if no_incluir.cota > mejor_valor:
            heapq.heappush(pq, no_incluir)
    
    print(f"Nodos explorados: {nodos_explorados}")
    print(f"Nodos podados: {nodos_podados}")
    
    return mejor_valor, mejor_solucion

# Prueba
pesos = [2, 3, 4, 5]
valores = [3, 4, 5, 8]
capacidad = 8

print("Mochila 0/1 con Branch and Bound")
print("="*60)
print(f"Capacidad: {capacidad}\n")
print("Items:")
for i in range(len(pesos)):
    print(f"  Item {i}: peso={pesos[i]}, valor={valores[i]}, ratio={valores[i]/pesos[i]:.2f}")

print(f"\n{'='*60}")

valor_max, items_sel = mochila_branch_bound(pesos, valores, capacidad)

print(f"\nValor máximo: {valor_max}")
print(f"Items seleccionados: {items_sel}")
print(f"\nDetalle:")
peso_total = 0
for i in items_sel:
    print(f"  Item {i}: peso={pesos[i]}, valor={valores[i]}")
    peso_total += pesos[i]
print(f"Peso total: {peso_total}/{capacidad}")

## Ejemplo 2: Problema del Viajante (TSP)

TSP simplificado usando Branch and Bound.

In [ ]:
import heapq

def tsp_branch_bound_simple(distancias):
    """
    TSP usando Branch and Bound (versión simplificada).
    Usa cota inferior simple: suma de aristas mínimas.
    """
    n = len(distancias)
    
    def calcular_cota_inferior(tour_actual, visitados):
        """Estima costo mínimo restante."""
        cota = 0
        
        # Costo del tour actual
        for i in range(len(tour_actual) - 1):
            cota += distancias[tour_actual[i]][tour_actual[i+1]]
        
        # Estimar costo restante
        no_visitados = [i for i in range(n) if i not in visitados]
        if no_visitados:
            # Mínima distancia desde última ciudad
            ultima = tour_actual[-1]
            min_salida = min(distancias[ultima][c] for c in no_visitados)
            cota += min_salida
            
            # Agregar aristas mínimas para ciudades restantes
            for ciudad in no_visitados:
                min_arista = min(distancias[ciudad][j] for j in range(n) if j != ciudad)
                cota += min_arista * 0.5  # Aproximación
        
        return cota
    
    mejor_costo = float('inf')
    mejor_tour = None
    nodos_explorados = 0
    
    # Cola: (cota, tour, visitados)
    pq = [(0, [0], {0})]
    
    while pq:
        cota, tour, visitados = heapq.heappop(pq)
        nodos_explorados += 1
        
        # Poda
        if cota >= mejor_costo:
            continue
        
        # Tour completo
        if len(tour) == n:
            costo_total = sum(distancias[tour[i]][tour[i+1]] 
                            for i in range(n-1))
            costo_total += distancias[tour[-1]][tour[0]]  # Volver al inicio
            
            if costo_total < mejor_costo:
                mejor_costo = costo_total
                mejor_tour = tour + [0]
            continue
        
        # Ramificar
        ciudad_actual = tour[-1]
        for siguiente in range(n):
            if siguiente not in visitados:
                nuevo_tour = tour + [siguiente]
                nuevo_visitados = visitados | {siguiente}
                nueva_cota = calcular_cota_inferior(nuevo_tour, nuevo_visitados)
                
                if nueva_cota < mejor_costo:
                    heapq.heappush(pq, (nueva_cota, nuevo_tour, nuevo_visitados))
    
    print(f"Nodos explorados: {nodos_explorados}")
    return mejor_tour, mejor_costo

# Prueba
distancias = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

print("Problema del Viajante (TSP)")
print("="*60)
print("Matriz de distancias:")
for fila in distancias:
    print(" ".join(f"{d:3d}" for d in fila))

print(f"\n{'='*60}\n")

tour, costo = tsp_branch_bound_simple(distancias)

print(f"\nMejor tour: {tour}")
print(f"Costo total: {costo}")

## 🎓 Conclusiones

- ✅ Branch & Bound encuentra solución óptima
- ✂️ La poda reduce drasticamente el espacio
- 📊 Cotas ajustadas = mejor poda
- 🎯 Best-First Search suele ser más eficiente

**Siguiente:** [Algoritmos Probabilísticos](../07_Algoritmos_Probabilisticos/README.md)